In [4]:
!pip install -q censusdata

In [5]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
from rasterio import plot, mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import matplotlib.pyplot as plt
import rasterstats
import rioxarray as rxr
from matplotlib_scalebar.scalebar import ScaleBar
from pathlib import Path
import xarray as xr
from shapely.geometry import Point, Polygon
import xyzservices.providers as xyz
import contextily as ctx
import censusdata
import seaborn as sns
import requests
import zipfile
import shutil
import requests
import rasterio.features 
from shapely.geometry import shape
import censusdata
from scipy.spatial import cKDTree
from rasterio.transform import from_origin
from rasterio.features import rasterize



## Get Census Data

Used to project select rasters to an aea crs that is centered in Spokane County

In [11]:
#Functions taken from lab 5
def get_census_data(tables, state, county, year=2019):
    '''Download census data for a given state and county fips code.'''

    # Download the data
    data = censusdata.download('acs5', year,  # Use 2019 ACS 5-year estimates
                               censusdata.censusgeo([('state', state), ('county', county), ('tract', '*')]),
                               list(tables.keys()))

    # Rename the column
    data.rename(columns=tables, inplace=True)

    # Extract information from the first column
    data['Name'] = data.index.to_series().apply(lambda x: x.name)
    data['SummaryLevel'] = data.index.to_series().apply(lambda x: x.sumlevel())
    data['State'] = data.index.to_series().apply(lambda x: x.geo[0][1])
    data['County'] = data.index.to_series().apply(lambda x: x.geo[1][1])
    data['Tract'] = data.index.to_series().apply(lambda x: x.geo[2][1])
    data.reset_index(drop=True, inplace=True)
    data = data[['Tract','Name']+list(tables.values())].set_index('Tract')
    
    return data

def get_census_tract_geom(state_fips, county_fips):
    '''Download census tract geometries for a given state and county fips code, storing in /tmp and cleaning up after.'''

    temp_dir = "/tmp/census_tracts"
    zip_path = os.path.join(temp_dir, f'tl_2019_{state_fips}_tract.zip')

    # Ensure temp directory exists
    os.makedirs(temp_dir, exist_ok=True)

    # Download the file
    url = f'https://www2.census.gov/geo/tiger/TIGER2019/TRACT/tl_2019_{state_fips}_tract.zip'
    response = requests.get(url, stream=True)
    if response.status_code != 200:
        raise Exception(f"Failed to download file: {url}")

    # Save ZIP file to temp directory
    with open(zip_path, "wb") as file:
        file.write(response.content)

    # Extract the ZIP file
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(temp_dir)

    # Find the shapefile in extracted contents
    for file in os.listdir(temp_dir):
        if file.endswith(".shp"):
            shapefile_path = os.path.join(temp_dir, file)
            break

    # Read the shapefile into a GeoDataFrame
    tracts = gpd.read_file(shapefile_path)

    # Filter by county and set index
    tracts = tracts[tracts['COUNTYFP'] == county_fips]
    tracts = tracts.rename(columns={'TRACTCE': 'Tract'}).set_index('Tract')

    # Cleanup: Remove extracted files and ZIP file
    shutil.rmtree(temp_dir)

    return tracts[['geometry']]



In [12]:
tables = {
'B01003_001E': 'TotalPopulation',
}

## FIPS Code for Washington
state_fips = '53' 

## FIPS code for Spokane County
county_fips = '063'  

census_df = get_census_data(tables, state_fips, county_fips)
tract_geom_gdf = get_census_tract_geom(state_fips, county_fips)
tract_geom_gdf_fo_proj = tract_geom_gdf



#### AEA Projection around Spokane

In [13]:
hull = tract_geom_gdf_fo_proj.geometry.unary_union.convex_hull
cent = hull.centroid

cent_lon =cent.x
cent_lat = cent.y
min_lat = tract_geom_gdf_fo_proj.bounds["miny"].min().item()
max_lat = tract_geom_gdf_fo_proj.bounds["maxy"].max().item()

proj_str_aea = f'+proj=aea +lat_1={min_lat:0.2f} +lat_2={max_lat:0.2f} +lat_0={cent_lat:0.2f} +lon_0={cent_lon:0.2f}'
print(proj_str_aea)

/tmp/ipykernel_158/4220196883.py:1: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  hull = tract_geom_gdf_fo_proj.geometry.unary_union.convex_hull


+proj=aea +lat_1=47.26 +lat_2=48.05 +lat_0=47.64 +lon_0=-117.42


# Land Cover

In [ ]:
land_url = " href here "

land_response = requests.get(land_url)

if land_response.status_code == 200:
    with open("Data/land_use.tif", "wb") as f:
        f.write(land_response.content)
    print("Land Cover Data Downloaded!")
else:
    print(f"Failed to fetch land data: {land_response.status_code}")

# Radiation

Uses the provided URL and bounding box to save a TIF file of the average raditaion in the area using NASA's POWER Map.

In [31]:
# Url from the Radiation ImageServer
server_url = "https://gis.earthdata.nasa.gov/image/rest/services/POWER/POWER_901_MONTHLY_RADIATION_UTC/ImageServer/exportImage"

# Spokane Box Boundary: (-117.823629, 47.259272, -117.039763, 48.047877)

# Defines the parameters for exporting an image
params = {
    "bbox": "-117.823629,47.259272,-117.039763,48.047877",  
    "bboxSR": "4326",
    "imageSR": "4326",
    "size": "100,100",  
    "format": "tiff",  
    "f": "json",
}


response = requests.get(server_url, params=params)

#Debugging code provided by ChatGPT
if response.status_code == 200:
    image_data = response.json()
    image_url = image_data.get("href", None)  # Extract image URL

    if image_url:
        print(f"Downloading image from: {image_url}")

        # Download the raster image
        image_response = requests.get(image_url)
        image_filename = "Data/radiation.tif"

        with open(image_filename, "wb") as file:
            file.write(image_response.content)

        print(f"Image saved as {image_filename}")
    else:
        print("No image URL found in response.")
else:
    print(f"Failed to fetch data: {response.status_code}, Response: {response.text}")


Image saved as Data/radiation.tif


# Substations

Using the URL, a geoson is downloaded containing the locations of all the substations in Spokane County.

In [29]:
url = "https://services6.arcgis.com/OO2s4OoyCZkYJ6oE/arcgis/rest/services/Substations/FeatureServer/0/query"

resolution = 100 

# Request parameters 
params = {
    "where": "COUNTY = 'SPOKANE'",  
    "outFields": "*", 
    "f": "geojson"  
}

response = requests.get(url, params=params)

#Debugging code provided by ChatGPT
# Check if the request was successful
if response.status_code == 200:
    # Load the GeoJSON into a GeoDataFrame
    substation_gdf = gpd.read_file(response.text)

else:
    print(f"Failed to fetch data: {response.status_code}")

# Substation Distance TIF

Generates a raster for which every pixel contains a value for the distance at that location to the nearsest substation.

In [30]:
# Code completed with the assistance of CHAT
substations = substation_gdf

# Reproject to the aea projection. It is fital to do this in the first step as projecting the raster later messes up the values for reasons we could not identify.
substations = substations.to_crs(epsg=32610)
substations = substations.to_crs(proj_str_aea)
tract_geom_gdf =tract_geom_gdf.to_crs(proj_str_aea)

# Get the bounding box from substations so that the raster will be the same dimensions as Spokane County
xmin, ymin, xmax, ymax = tract_geom_gdf.total_bounds

# Define the resolution and size of the raster
resolution = 100  
width = int((xmax - xmin) / resolution)
height = int((ymax - ymin) / resolution)


raster = np.full((height, width), np.nan, dtype=np.float32)

# Extract coordinates of substations
substation_coords = np.array(list(zip(substations.geometry.x, substations.geometry.y)))

# Create a spatial index using cKDTree for efficient distance computation
tree = cKDTree(substation_coords)

# Compute distances for each pixel
for i in range(height):
    for j in range(width):
        x = xmin + j * resolution + resolution / 2
        y = ymax - i * resolution - resolution / 2
        dist, _ = tree.query((x, y))
        raster[i, j] = dist / 1000  # converts it to km


transform = from_origin(xmin, ymax, resolution, resolution)

# Save raster to file
with rio.open(
    "./Data/substation_distance_raster.tif", "w", driver="GTiff", height=height, width=width,
    count=1, dtype=np.float32, crs=proj_str_aea, transform=transform
) as dst:
    dst.write(raster, 1)

### FloodPlain Bianary With Buffer

Opens the FEMA flood zone file of Spokane Washington creates a binary map of the flood plain in which 1 is not in the flood plain and 0 is. Also includes a 500 meter buffer between the two zones which is marked with the value 0.5.

In [18]:
floodplains_fn = './Data/FEMA_Flood_Zone.geojson'
floodplains_gdf = gpd.read_file(floodplains_fn)

floodplains_gdf = floodplains_gdf.set_crs(4326)
floodplains_gdf.head()

,OBJECTID,FLD_ZONE,FloodZone,FloodDescription,ShapeSTArea,ShapeSTLength,geometry
0,1,AE,100 Year,1 Percent Annual Chance Flood Zone,2563.373517,1057.303252,"POLYGON ((-117.21347 47.71921, -117.21342 47.7..."
1,2,AE,100 Year,1 Percent Annual Chance Flood Zone,47091.207776,1316.843587,"POLYGON ((-117.39354 47.5726, -117.3937 47.572..."
2,3,X,500 Year,0.2 Percent Annual Chance Flood Zone,12072.635706,1139.022057,"POLYGON ((-117.41693 47.66159, -117.41703 47.6..."
3,4,AE,100 Year,1 Percent Annual Chance Flood Zone,3293.344152,385.964150,"POLYGON ((-117.33046 47.92931, -117.33022 47.9..."
4,5,X,500 Year,0.2 Percent Annual Chance Flood Zone,140930.032756,2078.807675,"POLYGON ((-117.78801 47.42675, -117.78808 47.4..."


In [11]:

#Reproject now to aea to avoid repercussions later
floodplains_gdf = floodplains_gdf.to_crs(proj_str_aea)
tract_geom_gdf = tract_geom_gdf.to_crs(proj_str_aea)

# Get bounds of the county
xmin, ymin, xmax, ymax = tract_geom_gdf.total_bounds

# Define the resolution and size of the raster
resolution = 100  
width = int((xmax - xmin) / resolution)
height = int((ymax - ymin) / resolution)
transform = from_origin(xmin, ymax, resolution, resolution)

# For any value in the floodplains geometry, save it as 0. Then rasterize it using the desired height and width and any values not declared set to 1
shapes_floodplain = [(geom, 0) for geom in floodplains_gdf.geometry]
raster = rasterize(shapes_floodplain, out_shape=(height, width), transform=transform, fill=1, dtype=np.float32)

# Next, we created a buffer, and did the same thing, and instead of 0 set the values to be 0.5, keeping the fill 1
floodplains_gdf["geometry"] = floodplains_gdf.geometry.buffer(500)
shapes_buffer = [(geom, 0.5) for geom in floodplains_gdf.geometry]
buffer_raster = rasterize(shapes_buffer, out_shape=(height, width), transform=transform, fill=1, dtype=np.float32)

# Merges the rasters, keep original 0s, and overwriting the 1s with buffer values (0.5)
raster[(raster == 1) & (buffer_raster == 0.5)] = 0.5

# Save raster to file
with rio.open(
    "./Data/floodplain_with_buffer.tif", "w", driver="GTiff", height=height, width=width,
    count=1, dtype=np.float32, crs=proj_str_aea, transform=transform
) as dst:
    dst.write(raster, 1)
